# Data Engineer Challenge

## Importación de funciones

In [2]:
%load_ext memory_profiler

In [3]:
import os
from gcp import q1_bigquery
from q1_time import q1_time, q1_time_pandas
from q1_memory import q1_memory
from q2_time import q2_time, q2_time_pandas
from q2_memory import q2_memory
from q3_time import q3_time, q3_time_pandas
from q3_memory import q3_memory

# Enfoque general

A grandes rasgos por cada pregunta se realiza lo siguente:
 - Optimización de tiempo de ejecución cargando los datos en un DataFrame
     - Se usa la librería Pandas.
     - Se usa librería Polars, dada su capacidad de procesar grandes volúmenes de datos de forma rápida y eficiente.
 - Optimización de uso de memoría leyendo el JSON línea a línea.

La hipótesis es que para optimizar la velocidad resulta mejor cargar los datos a un Dataframe, ya sea de Pandas o Polars, debido a que están optimizados para trabajar de manera eficiente. De esta forma la manipulación de los datos y la aplicación de operaciones debería ser más rápida.

Por otra parte, resulta evidente que para optimizar el uso de memoria cargar todos los datos a un Dataframe no es el camino correcto. Resulta mucho más conveniente leer cada tweet, iterando el JSON linea a línea.

## Suposiciones
- Se presupone que se tiene el archivo JSON en la misma carpeta que este notebook.
- Se supone que el archivo ya está extraido del zip.
- Para ejecutar la función basada en BigQuery se asume que ya se tiene un proyecto creado, con una cuenta de servicio con permisos de administrador de BigQuery y de objetos de storage (y disponemos de la key de la cuenta en formato JSON).
- Se asume que se tiene un dataset en BigQuery con una tabla con los datos del JSON cargados.
- Se asume que tenemos las variables de entorno *PROJECT_ID*, *KEYFILE_PATH*, *DATASET_ID* y *TABLE_NAME* en el entorno virtual para usar BigQuery.

> En el notebook **setup_gcp.ipynb** se explica más en detalle el preparamiento para usar BigQuery

In [1]:
file_path = "farmers-protest-tweets-2021-2-4.json"

# Q1

Obtener las top 10 fechas con más tweets y mencionar el usuario con más publicaciones en cada uno de esos días.

## 1.1 Optimización de tiempo de ejecución

### 1.1.0 Enfoque usando BigQuery de Google Cloud Platform


- Se crea objeto Client de BigQuery a partir del *project_id* y el path de la *keyfile*.
- Se arma la query SQL, reemplazando el *project_id*, *dataset_id* y *table_name* según los argumentos entregados a la función.
- Se crea un BigQuery Job para ejecutar la query.
- Se entrega resultado como lista de tuplas.

In [5]:
project_id = os.getenv("PROJECT_ID")
keyfile_path = os.getenv("KEYFILE_PATH")
dataset_id = os.getenv("DATASET_ID")
table_name = os.getenv("TABLE_NAME")

#### Resultado de la función

In [6]:
print(q1_bigquery(keyfile_path, project_id, dataset_id, table_name))

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


#### Cálculo de tiempo de ejecución

In [7]:
%timeit q1_bigquery(keyfile_path, project_id, dataset_id, table_name)

2.62 s ± 117 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Cálculo de uso de memoría

In [9]:
%memit q1_bigquery(keyfile_path, project_id, dataset_id, table_name)

peak memory: 157.85 MiB, increment: 0.14 MiB


### 1.1.1 Enfoque usando librería Polars

- Se crea LazyFrame para leer el JSON.
- Se seleccionan de columnas a usar, tomando *date* y *username* (desde dentro de *user*).
- Se agrega columna *date_count* con conteo de tweets por día (usando lógica *over* similar a funciones de venta en SQL).
- Se agrega columna *user_count* con conteo tweets por combinacion de día y username (usando lógica *over*).
- Se ordena según *user_count*, se agrupa según *date* y se extrae el primer username por cada partición, junto con *date_count* asociada.
- Se ordena el resultado (una fila por día) por *date_count* descendiente y se extraen los primeros 10.
- Se materializa el LazyFrame a DataFrame.
- Se entrega resultado como lista de tuplas.

#### Resultado de la función

In [10]:
print(q1_time(file_path))

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


#### Cálculo de tiempo de ejecución

In [11]:
%timeit q1_time(file_path)

4.91 s ± 120 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Cálculo de uso de memoría

In [7]:
%memit q1_time(file_path)

peak memory: 1675.81 MiB, increment: 1524.47 MiB


### 1.1.2 Enfoque usando librería Pandas

- Se cargan de datos del JSON en un Dataframe.
- Se seleccionan de columnas a usar, tomando date y username (desde dentro de user).
- Se obtienen indices de los 10 días con más tweets y se filtran del dataframe.
- Se agrega columna *count* con conteo tweets por combinacion de día y username (usando *group_by*).
- Se obtienen los índices de usernames con más tweets por día haciendo *group_by* por día y extrayendo el índice de la fila con mayor *count* de cada partición.
- Se extraen los usernames del Dataframe según índices encontrados, se elimina columna con conteo y se entrega salida como lista de tuplas.

#### Resultado de la función

In [13]:
print(q1_time_pandas(file_path))

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 19), 'Preetm91'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria')]


#### Cálculo de tiempo de ejecución

In [14]:
%timeit q1_time_pandas(file_path)

8.65 s ± 445 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Cálculo de uso de memoría

In [8]:
%memit q1_time_pandas(file_path)

peak memory: 3247.39 MiB, increment: 2039.49 MiB


## 1.2 Optimización de uso de memoria

### Enfoque leyendo línea a línea

- Se lee JSON entero línea por línea, creando diccionario con cada día distinto como *key* y el conteo de tweets del día como *value*.
- Se ordena diccionario según conteo y se extraen los 10 días máximos.
- Por cada uno de los 10 días máximos:
    - Se lee JSON en línea por línea, creando diccionario con cada username como *key* y el conteo de tweets del user como *value*.
    - Se ordena diccionario y se extrae el username con más publicaciones del día (se agrega como tupla a una lista de salida).
- Se entrega lista resultante.

> Cabe mencionar que se está leyendo el JSON línea por línea 11 veces, lo que obviamente es lento, pero se hace debido al enfoque en uso de memoria.

> La idea es evitar crear un diccionario con dias como keys y diccionario de usernames como values, ya que esta variable ocuparía gran cantidad de memoría.
> Se prefiere iterar día a día extrayendo el username más mencionado, manteniendo en comparación una variable de menor tamaño que se sobreescribe en cada iteración.

#### Resultado de la función

In [16]:
print(q1_memory(file_path))

[(datetime.date(2021, 2, 12), 'RanbirS00614606'), (datetime.date(2021, 2, 13), 'MaanDee08215437'), (datetime.date(2021, 2, 17), 'RaaJVinderkaur'), (datetime.date(2021, 2, 16), 'jot__b'), (datetime.date(2021, 2, 14), 'rebelpacifist'), (datetime.date(2021, 2, 18), 'neetuanjle_nitu'), (datetime.date(2021, 2, 15), 'jot__b'), (datetime.date(2021, 2, 20), 'MangalJ23056160'), (datetime.date(2021, 2, 23), 'Surrypuria'), (datetime.date(2021, 2, 19), 'Preetm91')]


#### Cálculo de tiempo de ejecución

In [17]:
%timeit q1_memory(file_path)

1min 25s ± 818 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Cálculo de uso de memoría

In [4]:
%memit q1_memory(file_path)

peak memory: 148.39 MiB, increment: 1.01 MiB


## 1.3 Análisis de resultados

- En cuanto al tiempo de ejecución se puede ver que usar Polars es un 1.7 veces más rápido que Pandas y 17 veces más rápido que el enfoque en optimización de memoria. Es 0.53 veces más lento que BigQuery.
- En cuanto a uso de memoria, el enfoque leyendo línea a línea es 11 veces menor a usando Polars y 22 veces menor a usando Pandas. Ocupa aproximadamente la misma memoria que el enfoque usando BigQuery.

# Q2

Los top 10 emojis más usados con su respectivo conteo.

Para la realización de esta pregunta se utiliza la librería *emojis*.   
Se consideró la posibilidad de encontrar los emojis usando expresiones regulares debido a su mayor velocidad, pero se termina usando librería emojis para tener mayor precisión en los resultados.

> Se hace la suposición de que no se quieren agrupar emojis de la misma "forma" pero distinto color.
> Por ejemplo, corazón rojo y verde o *praying hands* de distinto tono de piel.

## 2.1 Optimización de tiempo de ejecución

### 2.1.1 Enfoque usando librería Polars

- Se crea LazyFrame para leer el JSON.
- Se seleccionan las columnas a usar, tomando *content* y aplicando una función para mapear *content* a una lista de emojis contenidos.
- Se filtran tweets sin emojis.
- Se materializa el LazyFrame a DataFrame.
- Se "abren" las filas creando una fila nueva por cada emoji en la lista.
- Se crea dataframe con conteo por emojis, se ordena de manera descendente y se extraen los 10 más usados.
- Se entrega resultado como lista de tuplas.

#### Resultado de la función

In [19]:
print(q2_time(file_path))

[('🙏', 5049), ('😂', 3072), ('🚜', 2972), ('🌾', 2182), ('🇮🇳', 2086), ('🤣', 1668), ('✊', 1651), ('❤️', 1382), ('🙏🏻', 1317), ('💚', 1040)]


#### Cálculo de tiempo de ejecución

In [20]:
%timeit q2_time(file_path)

21 s ± 181 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Cálculo de uso de memoría

In [21]:
%memit q2_time(file_path)

peak memory: 1034.31 MiB, increment: 386.59 MiB


### 2.1.2 Enfoque usando librería Pandas

- Se cargan de datos del JSON en un Dataframe.
- Se seleccionan las columnas a usar, tomando *content* y aplicando una función para mapear *content* a una lista de emojis contenidos.
- Se "abren" las filas creando una fila nueva por cada emoji en la lista.
- Se crea dataframe con conteo por emojis, se ordena de manera descendente y se extraen los 10 más usados.
- Se entrega resultado como lista de tuplas.

#### Resultado de la función

In [22]:
print(q2_time_pandas(file_path))

[(5049, '🙏'), (3072, '😂'), (2972, '🚜'), (2182, '🌾'), (2086, '🇮🇳'), (1668, '🤣'), (1651, '✊'), (1382, '❤️'), (1317, '🙏🏻'), (1040, '💚')]


#### Cálculo de tiempo de ejecución

In [23]:
%timeit q2_time_pandas(file_path)

26.7 s ± 532 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Cálculo de uso de memoría

In [9]:
%memit q2_time_pandas(file_path)

peak memory: 3620.69 MiB, increment: 2596.27 MiB


## 2.2 Optimización de uso de memoria

### Enfoque leyendo línea a línea

- Se crea un contador *Counter()* para los emojis.
- Se lee JSON entero línea por línea y por cada una:
    - Se extrae campo *content* y se extrae lista de emojis con librería *emojis*.
    - Por cada emoji en la lista se actualiza el contador de emojis.
- Se encuentran los 10 emojis más usados con método *most_common(10)* y se entregan como salida.

#### Resultado de la función

In [25]:
print(q2_memory(file_path))

[('🙏', 5049), ('😂', 3072), ('🚜', 2972), ('🌾', 2182), ('🇮🇳', 2086), ('🤣', 1668), ('✊', 1651), ('❤️', 1382), ('🙏🏻', 1317), ('💚', 1040)]


#### Cálculo de tiempo de ejecución

In [26]:
%timeit q2_memory(file_path)

24.2 s ± 670 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Cálculo de uso de memoría

In [5]:
%memit q2_memory(file_path)

peak memory: 149.76 MiB, increment: 1.38 MiB


## 2.3 Análisis de resultados

- En cuanto al tiempo de ejecución se puede ver que usar Polars es un 1.23 veces más rápido que Pandas y 1.14 veces más rápido que el enfoque en optimización de memoria.
- En cuanto a uso de memoria, el enfoque leyendo línea a línea es 6.9 veces menor a usando Polars y 24 veces menor a usando Pandas.

# Q3

Para la realización de esta pregunta se utiliza el campo *mentionedUsers* de los tweets.   
Se consideró la posibilidad de usar expresiones regulares haciendo uso del @, pero se prefirío usar el campo de usuarios mencionados, ya que probablemente entrega resultados más precisos (en teoría pueden haber @ no usados para referenciar a otros users, como en correos).

> Se hace la suposición de que por cada tweet se menciona máximo una vez a cada *username* distinto (en teoría se puede mencionar más de una vez en un mismo tweet a alguien). Dado que es algo que no ocurre demasiado, probablemente no afecta los resultados finales.

## 3.1 Optimización de tiempo de ejecución

### 3.1.1 Enfoque usando librería Polars

- Se crea LazyFrame para leer el JSON.
- Se seleccionan las columnas a usar, tomando *mentionedUsers* y aplicando una función para mapear *mentionedUsers* a una lista de *usernames* contenidos.
- Se filtran tweets sin menciones a otros usurios.
- Se materializa el LazyFrame a DataFrame.
- Se "abren" las filas creando una fila nueva por cada *username* en la lista.
- Se crea dataframe con conteo por *usernames*, se ordena de manera descendente y se extraen los 10 más mencionados.
- Se entrega resultado como lista de tuplas.

#### Resultado de la función

In [28]:
print(q3_time(file_path))

[('narendramodi', 2265), ('Kisanektamorcha', 1840), ('RakeshTikaitBKU', 1644), ('PMOIndia', 1427), ('RahulGandhi', 1146), ('GretaThunberg', 1048), ('RaviSinghKA', 1019), ('rihanna', 986), ('UNHumanRights', 962), ('meenaharris', 926)]


#### Cálculo de tiempo de ejecución

In [12]:
%timeit q3_time(file_path)

8.91 s ± 320 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Cálculo de uso de memoría

In [10]:
%memit q3_time(file_path)

peak memory: 1398.32 MiB, increment: 461.33 MiB


### 3.1.2 Enfoque usando librería Pandas

- Se cargan de datos del JSON en un Dataframe.
- Se seleccionan las columnas a usar, tomando *mentionedUsers* y aplicando una función para mapear *mentionedUsers* a una lista de *usernames* contenidos.
- Se "abren" las filas creando una fila nueva por cada *username* en la lista.
- Se crea dataframe con conteo por *usernames*, se ordena de manera descendente y se extraen los 10 más usados.
- Se entrega resultado como lista de tuplas.

#### Resultado de la función

In [31]:
print(q3_time_pandas(file_path))

[(2265, 'narendramodi'), (1840, 'Kisanektamorcha'), (1644, 'RakeshTikaitBKU'), (1427, 'PMOIndia'), (1146, 'RahulGandhi'), (1048, 'GretaThunberg'), (1019, 'RaviSinghKA'), (986, 'rihanna'), (962, 'UNHumanRights'), (926, 'meenaharris')]


#### Cálculo de tiempo de ejecución

In [13]:
%timeit q3_time_pandas(file_path)

8.1 s ± 156 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Cálculo de uso de memoría

In [11]:
%memit q3_time_pandas(file_path)

peak memory: 3311.57 MiB, increment: 2658.90 MiB


## 3.2 Optimización de uso de memoria

### Enfoque leyendo línea a línea

- Se crea un contador *Counter()* para los *usernames* mencionados.
- Se lee JSON entero línea por línea y por cada una:
    - Se extrae campo *mentionedUsers* y se extrae lista de *usernames*.
    - Por cada *username* en la lista se actualiza el contador de *usernames*.
- Se encuentran los 10 *usernames* más usados con método *most_common(10)* y se entregan como salida.

#### Resultado de la función

In [34]:
print(q3_memory(file_path))

[('narendramodi', 2265), ('Kisanektamorcha', 1840), ('RakeshTikaitBKU', 1644), ('PMOIndia', 1427), ('RahulGandhi', 1146), ('GretaThunberg', 1048), ('RaviSinghKA', 1019), ('rihanna', 986), ('UNHumanRights', 962), ('meenaharris', 926)]


#### Cálculo de tiempo de ejecución

In [35]:
%timeit q3_memory(file_path)

5.7 s ± 117 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Cálculo de uso de memoría

In [6]:
%memit q3_memory(file_path)

peak memory: 151.32 MiB, increment: 1.56 MiB


## 3.3 Análisis de resultados

- En cuanto al tiempo de ejecución se puede ver que el enfoque en optimización de memoria es 1.5 veces más rápido que Polars y 1.4 veces más rápido que Pandas.
- En cuanto a uso de memoria, el enfoque leyendo línea a línea es 9.2 veces menor a usando Polars y 22 veces menor a usando Pandas.

### 3.3.1 Conclusiones

- Se concluye que Polars es mucho más rápido para leer JSON separados por lineas gracias a su funcion scan_ndjson. El uso de LazyFrames realmente hace una diferencia comparado con la carga directa a un Dataframe de Pandas.

- Se ve que el uso de memoría usando Polars es significativamente menor al de Pandas.

- Se evidencia que cargar datos con estructura anidada y de un tamaño importante a un Dataframe no es una operación rápida.

- Resulta interesante que el enfoque para optimizar el uso de memoria, leyendo línea por linea resulta más rápido en varios casos. Probablemente la razón es el tiempo de carga de los datos de Polars y sobre todo de Pandas.

- El enfoque basado en BigQuery resulta mucho más rápido que el resto, pero la comparación es injusta, puesto que no se considera el tiempo de la carga de datos.

- En cuanto a la optimización del uso de memoria se puede ver que leer línea por linea supera consistentemente a la carga de los datos en un dataset, como uno esperaría teóricamente.

- Se puede ver que incluso la ejecución usando BigQuery (que debería ocupar muy poca memoria, ya que el trabajo lo hacen los servidores de Google) usa sobre 100MB, lo que indica que las funciones de optimización de memoría son muy eficientes, puesto que ocupan una memoría parecida.

- Se puede ver que los distintos enfoques entregan los mismos resultados, lo que hace probable que esten correctos.

### 3.3.2 Posibles mejoras

- Una posible mejora es preprocesar los datos para cargarlos más rápidamente en los Dataframes de Pandas y Polars. Resulta interesante usar el formato parquet para guardar los Dataframes dada su eficiencia y velocidad. Puesto que la mayor parte del tiempo de ejecución usando Pandas se gasta en la carga, resulta interesante esta opción para comparar en iguales condiciones con Polars.

- Una mejora sería usar librerías más completas (como cProfle) para evaluar el tiempo de ejecución, con el fin de descubrir posibles cuello de botella en las funciones. Sería una buena forma de confirmar la sospecha de los tiempo de carga en los Dataframe.

- Otra posible mejora es agregar el tiempo de carga de los datos en el caso de BigQuery, ya que con Pandas y Polars es el factor que más afecta.

- También sería bueno agregar mayor documentación sobre el setup de Google Cloud Platform, con cada paso explicado más detalladamente. 